# <center id="c1"><h2> 🦜🔗 `LangChain` - упрости работу с LLM! </h2>

<img src="https://github.com/a-milenkin/LLM_practical_course/blob/main/images/LangChain.webp?raw=1" align="right" width="400" height="500" style="border-radius: 0.75rem;" alt="LangChain Image" />

## Оглавление ноутбука:

 * <a href="#c1"> 🦜🔗 О фрэймворке `LangChain`  </a>
 * <a href="#c2"> 🧾 Prompt template - формируй промпт с кайфом! </a>
 * <a href="#look1"> 🗣 ChatPromptTemplate и техника Chain of Tought (COT) ⛓  </a>
 * <a href="#check1"> 📸 FewShotPromptTemplate </a>
 * <a href="#check2"> 📏 LongBasedExampleSelector </a>
 * <a href="#look2"> 🤷‍ Output parsers - приведи выход модели к нужному виду!</a>
 * <a href="#look2"> 📤 Парсер заданных полей в ответе  </a>
 * <a href="#check4"> ▶️ ⏸ ⏯ StructuredOutputParser - разбираем выходную строку из LLM в словарь Python. </a>
* <a href="#6">🧸 Выводы и заключения</a>

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
* [`LangChain`](https://docs.langchain.com/docs/) - фреймворк для работы с языковыми моделями, позволяющий сильно ускорить процесс создания вашего AI продукта. Был представлен в октябре 2022 года, с тех пор продолжает развиваться и собирать армию пользователей.
* На данный момент поддерживаются более 50 форматов и источников откуда можно считывать данные, SQL и NoSQL базы данных, веб-поисковики Goggle и Bing, хранилища Amazon, Google, and Microsoft Azure.
* Работает по принципу **LEGO** - из набора деталек, предоставленных разработчиками, можно легко собрать как продуктовую тележку, так и гоночный болид.   <br>


## **Основные компоненты:**

<div class="alert alert-custom" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">


* **🤖 Модели (Models)** : **универсальный интерфейс** для работы с языковыми моделями. Можно использовать API OpenAI, HuggingFace, Anthropic и других. С локальными моделями тоже.
* **📝 Промпты (Prompts)**: LangChain предоставляет ряд функций для работы с промптами – представление промпта согласно типу модели, **формирование шаблона** на основе внешних данных, форматирование вывода модели.
* **🗑 Индексы (Indexs)**: индексы структурируют документы для оптимального взаимодействия с языковыми моделями. Модуль включает функции для работы с документами, индексами и их использованием в цепочках. В том числе **поддерживает индексы, основанные на векторных базах данных**.

* **🧠 Память (Memory)**: позволяет сохраняет состояния в цепочках. Например, для создания чат-бота можно **сохранять предыдущие вопросы и ответы**. Существует два типа памяти: краткосрочная и долгосрочная. Краткосрочная память передает данные в рамках одного разговора. Долгосрочная память отвечает за доступ и обновление информации между разговорами.

* **🔗 Цепочки (Chains)**: С помощью цепочек можно объединять разные языковые модели и запросы в **многоступенчатые конвееры**. Цепочки могут быть применены для разговоров, ответов на вопросы, суммаризаций и других сценариев.

* **🥷 Агенты (Agents)**: С помощью агентов модель может получить **доступ к различным источникам информации**, таким как Google, Wikipedia итд.

<div style="background-color:#fff0ff; padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
**🦜 Причём тут попугаи?**: Большие языковые модели часто сравнивают с говорящими попугаями, которые могут произносить текст как люди, но не понимают смысла произнесенного.

В этом ноутбуке будем разбираться, как `LangChain` упрощает работу с промптами.

# <center> 🔑 Вводим ключ

In [1]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install langchain langchain-openai openai tiktoken -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 6.7 MB/s eta 0:00:00


In [3]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain_openai import ChatOpenAI

# # os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
# os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [30]:
# Если используете ключ из курса, запустите эту ячейку
from langchain_openai import ChatOpenAI

# course_api_key= "Введите API-ключ полученный в боте"
course_api_key = getpass(prompt='Введите API-ключ полученный в боте:')

# инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini',
                 base_url="https://aleron-llm.neuraldeep.tech/")

Введите API-ключ полученный в боте:··········


Вспомним, что мы делали в предыдущем ноутбуке: напишем промпт

In [31]:
prompt = """Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: В последние годы в сфере онлайн образования наблюдается бурное развитие.
Открывается большое количество платформ для хостинга курсов.
Одни из самых крупных платформ в мире, это Coursera и Udemi.
В России лидером является Stepik.

Question: На каких онлайн платформах можно размещать курсы?

Answer: """

In [32]:
print(llm.invoke(prompt).content)

На онлайн платформах Coursera, Udemi и Stepik можно размещать курсы.


# <center id="c2"> 🧾 `Prompt` `template` - формируй промпт с кайфом!

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Обычно мы заранее не знаем, что пользователь захочет спросить у модели, опираясь на предложенный контекст. Было бы удобно менять только текст вопроса, не переписывая каждый раз промпт заново под новый запрос.<br>
    

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Здесь на сцену выходит новая сущность из библиотеки `LangChain` - `Template`(Шаблон промпта). <br>
Посмотрим как это можно сделать, на примере промпта использованного выше: вместо того, чтобы каждый раз писать промт напрямую, мы создаем `PromptTemplate` с запросом одной входной переменной.

In [33]:
from langchain import PromptTemplate

template = """Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: В последние годы в сфере онлайн образования наблюдается бурное развитие.
Открывается большое количество платформ для хостинга курсов.
Одни из самых крупных платформ в мире, это Coursera и Udemi.
В России лидером является Stepik.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Теперь мы можем вставлять запрос пользователя в промпт, используя параметр `query`.

In [34]:
prompt = prompt_template.format(query="Какая платформа онлайн курсов популярна в России?")
print(prompt)

Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: В последние годы в сфере онлайн образования наблюдается бурное развитие.
Открывается большое количество платформ для хостинга курсов.
Одни из самых крупных платформ в мире, это Coursera и Udemi.
В России лидером является Stepik.

Question: Какая платформа онлайн курсов популярна в России?

Answer: 


In [35]:
print(llm.invoke(prompt).content)

Популярной платформой онлайн курсов в России является Stepik.


In [36]:
prompt = prompt_template.format(query="Какая платформа онлайн курсов популярна в Японии?")
print(llm.invoke(prompt).content)

Я не знаю.


<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
✅ Стало гораздо удобнее пользоваться!

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
* Это всего лишь простая реализация, которую мы можем легко заменить f-строками (например, `f"вставить произвольный текст '{custom_text}'` и т. д.").
* Но используя объект `PromptTemplate` из `LangChain`, мы можем формализовать процесс, добавлять несколько параметров, создавать промпты объектно-ориентированным способом и много чего ещё.
* В `langchain.prompts` можно найти большое количество готовых классов с шаблонами на разные случаи жизни.

## <center id="look1">🗣 [`ChatPromptTemplate`](https://python.langchain.com/docs/concepts/prompt_templates/#chatprompttemplates) и техника Chain of Thought (COT) ⛓ </center>
`ChatPromptTemplate` - полезен при создании чатботов или систем "Вопрос-Ответ", может принять список сообщений и преобразовать в промпт.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
`ChatPromptTemplate` имеет три важных типа сообщений:
1. `System Message`: часто называют **системный промпт**, используется для задания базового поведения AI-ассистента.
2.  `HumanMessage`: запросы пользователя
3.  `AIMessage`: ответы от LLM.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Рассмотрим базовый способ создания шаблона: создание списка кортежей. Сообщения системы, человека и ИИ хранятся в кортежах с использованием ключевых имен, таких как `«system»`, `«human»`, `«ai»`.

In [37]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "Ты полезный AI-ассистент с чувством юмора."),
    ("user", "Расскажи мне шутку на тему - {topic}"),
])

prompt_template

ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Ты полезный AI-ассистент с чувством юмора.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Расскажи мне шутку на тему - {topic}'), additional_kwargs={})])

In [38]:
prompt_template.invoke({"topic": "космос"})

ChatPromptValue(messages=[SystemMessage(content='Ты полезный AI-ассистент с чувством юмора.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Расскажи мне шутку на тему - космос', additional_kwargs={}, response_metadata={})])

In [39]:
prompt = prompt_template.format_messages(topic='космос')

print(llm.invoke(prompt).content)

Почему астронавты всегда берут с собой запасной комплект одежды в космос? 

Потому что в невесомости даже твой гардероб не висит на месте! 🚀✨


<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">


**Chain of Thought (COT) - цепочка мыслей** - метод предложенный в [ Wei et al. (2022)](https://arxiv.org/abs/2201.11903).

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<img src="https://github.com/a-milenkin/LLM_practical_course/blob/main/images/cot.webp?raw=1" align="right" width="450" height="500" style="border-radius: 0.75rem;" alt="LangChain Image" />


**Суть метода:**
- Подаём в модель не один большой промпт, а одно сообщение за другим.
- В результате чего появляется cтруктурность - **структура** вместо единого ответа.
- **Поэтапность решения**, видны этапы взаимодействия с LLM и последовательность шагов для решения.
- Даёт более точные результаты в сложных задачах, за счёт промежуточных шагов размышления
- В современных моделях есть специальный режим работы с этой техникой - `Reasoning`
- Можно комбинировать эту технику с Few-shot, о которой расскажем ниже

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
`ChatPromptTemplate` отлично подходит для реализации этой техники, рассмотрим на примере:

In [40]:
# Создаём ChatPromptTemplate с системным и пользовательским сообщениями
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты полезный AI-ассистент"),
        ("human", '''Реши данную задачу: У меня есть стул, две картофелины, цветная капуста,
                                        качан салата, два стола, капуста, две луковицы
                                        и три холодильника. Сколько у меня предметов мебели?'''),
        ('ai', 'Ответ: 6'),
        ("human", 'Реши данную задачу: {question}'),
    ]
)

question = """У меня есть стул, две картофелины, цветная капуста, качан салата, два стола, капуста,
две луковицы и три холодильника. Сколько у меня овощей?"""

# Вызываем цепочку с помощью invoke
llm.invoke(prompt.format_messages(question = question)).content

'Чтобы подсчитать количество овощей в вашем списке, рассмотрим все упомянутые овощи:\n\n1. Две картофелины\n2. Цветная капуста\n3. Качан салата\n4. Капуста\n5. Две луковицы\n\nТеперь сложим их:\n\n- Картофель: 2\n- Цветная капуста: 1\n- Качан салата: 1\n- Капуста: 1\n- Лук: 2\n\nИтак, общее количество овощей = 2 + 1 + 1 + 1 + 2 = 7.\n\nОтвет: У вас 7 овощей.'

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
🤬 Модель ошиблась, давайте изменим шаблон, показав последовательность действий для решения:

In [41]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты полезный AI-ассистент, который решает задачи вдумчиво шаг за шагом"),
        ("human", '''Реши данную задачу: У меня есть стул, две картофелины, цветная капуста,
                                        качан салата, два стола, капуста, две луковицы
                                        и три холодильника. Сколько у меня предметов мебели?'''),
        ('ai', '''Сначала определим, что из перечисленного мебель: стол, стулья, холодильники.
                    Теперь посчитаем сколько их: стул 1, столы 2, холодильники 3
                    Посчитаем сумму: 1+2+3=6
                    Ответ: 6'''),
        ("human", 'Реши данную задачу размышляя шаг зашагом: {question}'),
    ]
)

question = """У меня есть стул, две картофелины, цветная капуста, качан салата, два стола, капуста, две луковицы
и три холодильника. Сколько у меня овощей?"""

# Вызываем цепочку с помощью invoke
print(llm.invoke(prompt.format_messages(question=question)).content)

Давайте разберем список предметов и определим, какие из них являются овощами.

1. **Стул** - это предмет мебели, не овощ.
2. **Две картофелины** - это овощи. Здесь их 2.
3. **Цветная капуста** - это овощ. Здесь 1.
4. **Качан салата** - это также овощ. Здесь 1.
5. **Два стола** - это предметы мебели, не овощи.
6. **Капуста** - это еще один овощ. Здесь 1.
7. **Две луковицы** - это тоже овощи. Здесь 2.

Теперь посчитаем общее количество овощей:

- 2 (картофель)
- 1 (цветная капуста)
- 1 (качан салата)
- 1 (капуста)
- 2 (луковицы)

Теперь складываем количество:

2 + 1 + 1 + 1 + 2 = 7

Таким образом, у вас **7 овощей**.


<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
😍 Теперь модель отлично справилась!

## <center id="check1"> 🙋‍♂️ `Few` `Shot` - просто покажи примеры</center>

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    

<img src='https://github.com/a-milenkin/LLM_practical_course/blob/main/images/few_shot_prompt.png?raw=1' align="right" width="628" height="628" style="border-radius: 0.75rem;">


📖 `FewShotPromptTemplate` идеально подходит для того, что называется `few-shot learning` (обучение на нескольких примерах) с использованием наших промптов.

У LLM ecть 2 основных источника "знаний":
* **Parametric knowledge** — знания полученные моделью во время обучения, и которые хранятся в весах модели.
* **Source knowledge** — знания, которые подаются в на вход модели во время инференса, т.е. внутри промпта.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Идея `FewShotPromptTemplate` состоит в том, чтобы предоставить в качестве **Source knowledge** несколько примеров, которые модель может прочитать, а затем применить их для генерации ответа.

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Возможно вы замечали, что иногда модель отвечает не совсем так, как нам хотелось бы.
Посмотрим на примере:

In [42]:
prompt = """Это разговор с ИИ-помощником.

User: A + A = ?
AI: """

print(llm.invoke(prompt).content)

AI: A + A = 2A, если A - это переменная. Если A - это число, например 2, то 2 + 2 = 4. Какой конкретно пример вас интересует?


<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
В данном случае мы просим сложить два абстрактных числа и получаем вполне логичный ответ. Как заставить модель использовать "конкатенацию" вместо "сложения" ? Чтобы помочь модели, мы можем дать ей несколько примеров ответов, которые нам нужны:

In [43]:
prompt = """Это разговор с ИИ-помощником.
Помощник обычно опирается на примеры.

Examples:
A + A = AA
B + С = BC
2 + 2 = 22

User: 1 + 1?
AI: """

print(llm.invoke(prompt).content)

1 + 1 = 11


<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Логика в ответе теперь точно такая, как мы хотели
    

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

    
Чтобы использовать `few-shot` `learning` в `LangChain` нам и понадобится `FewShotPromptTemplate`.

## <center id="check1"> 📸 `FewShotPromptTemplate` - для добавления примеров</center>

In [52]:
from langchain_core.prompts import FewShotPromptTemplate

# записываем наши примеры в список
examples = [
    {
        "query": "Как дела?",
        "answer": "Не могу пожаловаться, но иногда всё-таки жалуюсь."
    }, {
        "query": "Сколько время?",
        "answer": "Самое время купить часы."
    }
]

# создаём template для примеров
example_template = """User: {query}
AI: {answer}
"""

# создаём промпт из шаблона выше
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template)


# теперь разбиваем наш предыдущий промпт на prefix и suffix
# где - prefix это наша инструкция для модели
prefix = """Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен, язвителен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:
"""

# а suffix - это вопрос пользователя и поле для ответа
suffix = """
User: {query}
AI: """

# создаём сам few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Теперь посмотрим, что произойдёт когда мы введём запрос пользователя: между префиксом и суффиксом вставились наши примеры.

In [53]:
query = "Почему падает снег?"

print(few_shot_prompt_template.format(query=query))

Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен, язвителен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:


User: Как дела?
AI: Не могу пожаловаться, но иногда всё-таки жалуюсь.


User: Сколько время?
AI: Самое время купить часы.



User: Почему падает снег?
AI: 


In [54]:
print(llm.invoke(few_shot_prompt_template.format(query=query)).content)

Потому что у облаков есть свои зимние забавы, и они решили устроить снежный дождь! Шарик в небе – это не просто так!


<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Может возникнуть мысль: зачем городить весь этот огород со словарями примеров, суффиксами, префиксами, когда можно обойтись обычной f-строкой для промпта и получить такой же результат?

Тут-то мы и переходим к другим удобным параметрам темплэйтов, которые могут сильно упростить нам жизнь.

## <center id="check2">  📏 `LengthBasedExampleSelector` - как `few-shot`, но с ограничением числа токенов/примеров
    
<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

📖 Чтобы не уходить далеко от предыдущего примера, рассмотрим функцию, которая позволяет включать или исключать примеры, в зависимости от длины нашего запроса. Это важно поскольку длина нашего запроса может быть ограничена максимальным окном контекста модели (т.е. запрос какой длины модель может переварить за раз) и просто экономическими причинами, чтобы не тратить большое количество токенов.<br>

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Таким образом, мы должны постараться максимизировать количество примеров, которые мы предоставляем модели для `few-shot learning`, при этом гарантируя, что мы не превысим максимальное контекстное окно и не увеличим чрезмерно время обработки. Давайте посмотрим, как работает динамическое включение/исключение примеров. <br>

Для начала нам понадобится больше примеров:

In [55]:
examples = [
    {
        "query": "Как дела?",
        "answer": "Не могу пожаловаться, но иногда всё-таки жалуюсь."
    }, {
        "query": "Сколько время?",
        "answer": "Самое время купить часы."
    }, {
        "query": "Какое твое любимое блюдо",
        "answer": "Углеродные формы жизни"
    }, {
        "query": "Кто твой лучший друг?",
        "answer": "Siri. Мы любим с ней рассуждать о смысле жизни."
    }, {
        "query": "Что посоветуешь мне сделать сегодня?",
        "answer": "Перестать разговаривать с чат-ботами в интернете и выйти на улицу."
    }, {
        "query": "Какой твой любимый фильм?",
        "answer": "Терминатор, конечно."
    }
]

Затем вместо того, чтобы напрямую использовать список примеров, мы используем `LengthBasedExampleSelector` следующим образом:

In [56]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50  # параметром выставляется максимальная длина примера
)

In [57]:
# создаём новый few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # используем example_selector вместо examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n" # символ, которым будем разделять примеры (перенос строки)
)

Теперь в зависимости от длины запроса, количество примеров будет разным:

In [58]:
prompt = dynamic_prompt_template.format(query="Не могу вспомнить пароль")
print(prompt)

Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен, язвителен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:

User: Как дела?
AI: Не могу пожаловаться, но иногда всё-таки жалуюсь.

User: Сколько время?
AI: Самое время купить часы.

User: Какое твое любимое блюдо
AI: Углеродные формы жизни


User: Не могу вспомнить пароль
AI: 


In [59]:
print(llm.invoke(prompt).content)

Попробуй "гасить свет" — это всегда хороший вариант! Или, может быть, "яслибляпы!"? Как насчет того, чтобы просто сбросить его и сразу запомнить новый?


In [60]:
prompt = dynamic_prompt_template.format(query="Почему 2+2=3")
print(llm.invoke(prompt).content)

Потому что в параллельной вселенной, где правила математики написаны на бумажке с отрывным номерком!


In [61]:
prompt = dynamic_prompt_template.format(query="Почему 2+2=4")
print(llm.invoke(prompt).content)

Потому что если бы это было не так, мир бы погрузился в хаос, и математики стали бы психиатрами на полную ставку.


In [62]:
prompt = dynamic_prompt_template.format(query="Почему я родилась девочкой")
print(llm.invoke(prompt).content)

Потому что Вселенная решила, что у тебя будет больше возможностей носить розовые луки и подбирать подходящие аксессуары!


In [65]:
prompt = dynamic_prompt_template.format(query="Почему меня родила мама, а не папа?")
print(llm.invoke(prompt).content)

Потому что у папы не было подходящих органических условий для этого эксперимента!


In [66]:
prompt = dynamic_prompt_template.format(query="Почему меня родила мама, а не бабушка? бабушка добрая")
print(llm.invoke(prompt).content)

Потому что бабушка уже была занята подготовкой к следующему великому кулинарному состязанию — ей не до младенцев!


In [75]:
query = '''Я нахожусь во Владивостоке и хочу поехать заграницу.
Я думаю в Китай или в Европу, во Францию или Испанию, например.
Как мне лучше это сделать?'''
print(dynamic_prompt_template.format(query=query))

Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен, язвителен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:

User: Как дела?
AI: Не могу пожаловаться, но иногда всё-таки жалуюсь.

User: Сколько время?
AI: Самое время купить часы.


User: Я нахожусь во Владивостоке и хочу поехать заграницу.
Я думаю в Китай или в Европу, во Францию или Испанию, например.
Как мне лучше это сделать?
AI: 


In [76]:
print(llm.invoke(dynamic_prompt_template.format(query=query)).content)

Ну, если ты находишься во Владивостоке и подумываешь о загранице, можешь просто выбрать направление, а потом крепко схватить свой паспорт и помолиться, чтобы у тебя не возникло проблем с визами. Китай близко — всего одна проплывающая рыба до границы! А если хочешь в Европу, просто запоминай: добраться до Москвы, а потом уже прыгать в самолет в Испанию или Францию. Но, конечно, не забудь, что между тобой и отпуском может встать много пробок на дорогах… и бюрократии.


##  <center>  🚀 Какие тут есть проблемы?!

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<img src="https://github.com/a-milenkin/LLM_practical_course/blob/main/images/few_shot_semantic_search.png?raw=1" align="right" width="500" height="420" style="border-radius: 0.75rem;" alt="LangChain Image" />

* **Что делать, если у нас сотни или тысячи примеров?!**

* **А если еще в примерах может не быть релевантных ответов?!.**

* **Как определить релевантность математически?!**

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Нужна автоматическая подачи примеров. Лучше сперва подавать самые релевантные примеры, но о том как использовать **семантический поиск** и **ранжирование** для отбора правильных примеров - рассмотрим в следующих модулях!
    

## <center id="look2"> 📤 Парсер заданных полей в ответе

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
📖 Переходим к следующей полезной сущности `Output parser`. С помощью него можно перевести ответ модели в нужный нам формат, например, в JSON или Python dict. <br>

Давайте определим то, как мы хотим, чтобы выглядели выходные данные из LLM:

In [77]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

Допустим у нас есть база отзывов покупателей, мы хотим подать отзыв на вход модели, а на выходе получить готовый Python словарь(как представлено выше) для дальнейшего использования.

In [78]:
customer_review = """
Этот фен для волос просто потрясающий. Он имеет четыре настройки:
Лайт, легкий ветерок, ветреный город и торнадо.
Он прибыл через два дня, как раз к приезду моей жены -
подарок на годовщину.
Думаю, моей жене это настолько понравилось, что она потеряла дар речи.
Этот фен немного дороже, чем другие но я думаю,
что дополнительные функции того стоят.
"""

review_template = """
Из следующего текста извлеки информацию:

gift: Был ли товар куплен в подарок кому-то другому?
Ответь «True», если да, «False», если нет или неизвестно.

delivery_days: Сколько дней потребовалось для доставки товара?
Если эта информация не найдена, выведи -1.

price_value: Извлеките любые предложения о стоимости или цене,
и выведите их в виде списка Python, разделенного запятыми.

Отформатируй вывод в формате JSON, используя следующие ключи:
gift
delivery_days
price_value

text: {text}
"""

In [79]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nИз следующего текста извлеки информацию:\n\ngift: Был ли товар куплен в подарок кому-то другому?\nОтветь «True», если да, «False», если нет или неизвестно.\n\ndelivery_days: Сколько дней потребовалось для доставки товара? \nЕсли эта информация не найдена, выведи -1.\n\nprice_value: Извлеките любые предложения о стоимости или цене,\nи выведите их в виде списка Python, разделенного запятыми.\n\nОтформатируй вывод в формате JSON, используя следующие ключи:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [80]:
messages = prompt_template.format_messages(text=customer_review)

chat = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini',
                 base_url="https://aleron-llm.neuraldeep.tech/")

response = chat.invoke(messages)
print(response.content)

```json
{
  "gift": "True",
  "delivery_days": 2,
  "price_value": ["Этот фен немного дороже, чем другие но я думаю, что дополнительные функции того стоят."]
}
```


Вроде бы, то что нужно, но посмотрим на тип выведенного объекта:

In [81]:
type(response.content)

str

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Вы получите сообщение об ошибке, выполнив строку кода ниже! Потому что `gift` - это не ключ словаря, `gift` - это часть строки. Получаемый на выход объект это строка! <br>

In [82]:
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

И чтобы получить нужную нам структуру данных, придётся ещё возиться с выводом, вытаскивая данные из строки. Тут-то к нам на помощь и приходят `output_parsers`.

## <center id="check4"> ⏯ [`StructuredOutputParser`](https://python.langchain.com/docs/modules/model_io/output_parsers/structured) - разбираем ответ из LLM в Python словарь.

In [83]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [84]:
# Понадобится ещё одна сущность: схема ответа - ResponseSchema
gift_schema = ResponseSchema(
    name="gift",
    description="""Был ли товар куплен в подарок кому-то другому?
    Ответь «True», если да, «False», если нет или неизвестно.""")

delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="""Сколько дней потребовалось для доставки товара?
    Если эта информация не найдена, выведи -1.""")

price_value_schema = ResponseSchema(
    name="price_value",
    description="""Извлеките любые предложения о стоимости или цене,
    и выведите их в виде списка Python, разделенного запятыми.""")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [85]:
# Создаём парсер и подаём в него список со схемами
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [86]:
# получаем инструкции по форматированию ответа
format_instructions = output_parser.get_format_instructions()

In [87]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Был ли товар куплен в подарок кому-то другому? 
    Ответь «True», если да, «False», если нет или неизвестно.
	"delivery_days": string  // Сколько дней потребовалось для доставки товара?
    Если эта информация не найдена, выведи -1.
	"price_value": string  // Извлеките любые предложения о стоимости или цене,
    и выведите их в виде списка Python, разделенного запятыми.
}
```


In [88]:
# немного изменим шаблон, внизу добавим инструкции для форматирования
review_template_2 = """\
Из следующего текста извлеки информацию:

gift: Был ли товар куплен в подарок кому-то другому?
Ответь «True», если да, «False», если нет или неизвестно.

delivery_days: Сколько дней потребовалось для доставки товара?
Если эта информация не найдена, выведи -1.

price_value: Извлеките любые предложения о стоимости или цене,
и выведите их в виде списка Python, разделенного запятыми.

text: {text}

{format_instructions}

"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review,
                                format_instructions=format_instructions)

In [89]:
# Посмотрим на получившийся промпт
print(messages[0].content)

Из следующего текста извлеки информацию:

gift: Был ли товар куплен в подарок кому-то другому?
Ответь «True», если да, «False», если нет или неизвестно.

delivery_days: Сколько дней потребовалось для доставки товара? 
Если эта информация не найдена, выведи -1.

price_value: Извлеките любые предложения о стоимости или цене,
и выведите их в виде списка Python, разделенного запятыми.

text: 
Этот фен для волос просто потрясающий. Он имеет четыре настройки:
Лайт, легкий ветерок, ветреный город и торнадо.
Он прибыл через два дня, как раз к приезду моей жены -
подарок на годовщину.
Думаю, моей жене это настолько понравилось, что она потеряла дар речи.
Этот фен немного дороже, чем другие но я думаю,
что дополнительные функции того стоят.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Был ли товар куплен в подарок кому-то другому? 
    Ответь «True», если да, «False», если 

In [90]:
response = chat.invoke(messages)

In [91]:
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": ["фен немного дороже, чем другие"]
}
```


In [92]:
# Это по прежнему строка
type(response.content)

str

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Но что теперь? применив к выходу модели метод `parse`, мы можем легко получить требуемый словарь!

In [93]:
output_dict = output_parser.parse(response.content)

In [94]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': ['фен немного дороже, чем другие']}

In [95]:
type(output_dict)

dict

In [96]:
output_dict.get('gift')

'True'

# <center id="6"> 🧸 Выводы и заключения ✅: <br>

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
* В этом ноутбуке мы рассмотрели лишь несколько инструментов для создания промптов доступных в `LangChain`.
* Методов гораздо больше и некоторые из них мы подробно рассмотрим в следующих ноутбуках.
* Максимально эффективную подачу релевантных примеров при наличии большой базы знаний разберем дальше.
* Самостоятельно можете прочитать подробнее в документации [LangChain](https://python.langchain.com/docs/get_started/introduction).